# micrograd exercises

In [1]:
from __future__ import annotations

from math import cos, exp, log, sin

import torch
import torch.nn.functional as F


## Section 1: derivatives

In [2]:
# Here is a mathematical expression that takes 3 inputs and produces one output.


def f(a: float | int, b: float | int, c: float | int) -> float:
    return -(a**3) + sin(3 * b) - 1.0 / c + b**2.5 - a**0.5


print(f(2, 3, 4))


6.336362190988558


Write the function df that returns the analytical gradient of f, i.e., use your skills from calculus to take the derivative, then implement the formula.

In [3]:
def gradf(a: float | int, b: float | int, c: float | int) -> list[float]:
    dfda = -3 * a**2 - 1 / (2 * a**0.5)
    dfdb = 3 * cos(3 * b) + 2.5 * b**1.5
    dfdc = 1 / c**2
    return [dfda, dfdb, dfdc]


# Expected answer is the list below
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
    ok = "OK" if abs(yours[dim] - ans[dim]) < 1e-5 else "WRONG!"
    print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


Now estimate the gradient numerically without any calculus, using the approximation we used in the video. You should not call the function ``gradf`` from the last cell.

In [4]:
h = 1e-8

dfda_approx = (f(2 + h, 3, 4) - f(2, 3, 4)) / h
dfdb_approx = (f(2, 3 + h, 4) - f(2, 3, 4)) / h
dfdc_approx = (f(2, 3, 4 + h) - f(2, 3, 4)) / h

numerical_grad = [dfda_approx, dfdb_approx, dfdc_approx]

for dim in range(3):
    ok = "OK" if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else "WRONG!"
    print(
        f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}"
    )


OK for dim 0: expected -12.353553390593273, yours returns -12.353553380251014
OK for dim 1: expected 10.25699027111255, yours returns 10.256990368162633
OK for dim 2: expected 0.0625, yours returns 0.0624999607623522


There is an alternative formula that provides a much better numerical approximation to the derivative of a function. Learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative. Implement it. Confirm that for the same step size h this version gives a better approximation.

In [5]:
h = 1e-8

dfda_approx = (f(2 + h, 3, 4) - f(2 - h, 3, 4)) / (2 * h)
dfdb_approx = (f(2, 3 + h, 4) - f(2, 3 - h, 4)) / (2 * h)
dfdc_approx = (f(2, 3, 4 + h) - f(2, 3, 4 - h)) / (2 * h)

numerical_grad2 = [dfda_approx, dfdb_approx, dfdc_approx]

for dim in range(3):
    ok = "OK" if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else "WRONG!"
    print(
        f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}"
    )


OK for dim 0: expected -12.353553390593273, yours returns -12.353553291433172
OK for dim 1: expected 10.25699027111255, yours returns 10.256990368162633
OK for dim 2: expected 0.0625, yours returns 0.0624999607623522


In [6]:
# Comparison of the two approximations
for dim in range(3):
    one_sided = abs(numerical_grad[dim] - ans[dim])
    symmetric = abs(numerical_grad2[dim] - ans[dim])
    print("Dim", dim)
    print("One-sided approximation:", one_sided)
    print("Symmetric approximation:", symmetric)
    print("Symmetric - one-sided: ", symmetric - one_sided)


Dim 0
One-sided approximation: 1.0342258605078314e-08
Symmetric approximation: 9.916010057509084e-08
Symmetric - one-sided:  8.881784197001252e-08
Dim 1
One-sided approximation: 9.705008352511868e-08
Symmetric approximation: 9.705008352511868e-08
Symmetric - one-sided:  0.0
Dim 2
One-sided approximation: 3.923764779756311e-08
Symmetric approximation: 3.923764779756311e-08
Symmetric - one-sided:  0.0


It actually seems worse here. :) But it is generally a better approximation.

# Section 2: support for softmax

In [7]:
# Value class starter code, with many functions taken out
class Value:
    def __init__(
        self, data: float | int, label: str = "", _prev: tuple = (), _op: str = ""
    ) -> None:
        self.data = data
        self.grad = 0.0
        self.label = label
        self._backward = lambda: None
        self._prev = set(_prev)
        self._op = _op

    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other: Value | float | int) -> Value:  # Exactly as in the video
        other = other if isinstance(other, Value) else Value(data=other)
        out = Value(data=self.data + other.data, _prev=(self, other), _op="+")

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad

        out._backward = _backward

        return out

    # Re-implement all the other functions needed for the exercises below

    def __radd__(self, other: Value | float | int) -> Value:
        return self + other

    def __mul__(self, other: Value | float | int) -> Value:
        other = other if isinstance(other, Value) else Value(data=other)
        out = Value(data=self.data * other.data, _prev=(self, other), _op="*")

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad

        out._backward = _backward

        return out

    def __rmul__(self, other: Value | float | int) -> Value:
        return self * other

    def __neg__(self) -> Value:
        return self * -1

    def __truediv__(self, other: Value | float | int) -> Value:
        return self * other**-1

    def __pow__(self, other: float | int) -> Value:
        if not isinstance(other, (float, int)):
            raise ValueError("Only float or int exponents are supported.")

        def _backward():
            self.grad += other * self.data ** (other - 1) * out.grad

        out = Value(data=self.data**other, _prev=(self,), _op=f"**{other}")
        out._backward = _backward

        return out

    def exp(self) -> Value:
        def _backward():
            self.grad += out.data * out.grad

        out = Value(data=exp(self.data), _prev=(self,), _op="exp")
        out._backward = _backward

        return out

    def log(self) -> Value:
        def _backward():
            self.grad += 1 / self.data * out.grad

        out = Value(data=log(self.data), _prev=(self,), _op="log")
        out._backward = _backward

        return out

    def backward(self) -> None:
        topo = []
        visited = set()

        def build_topo(node: Value) -> None:
            if node not in visited:
                visited.add(node)
                for child in node._prev:
                    build_topo(child)
                topo.append(node)

        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()


Without referencing our code/video __too__ much, make this cell work. You'll have to implement (in some cases re-implement) a number of functions of the Value object, similar to what we've seen in the video. Instead of the squared error loss, this implements the negative log-likelihood loss, which is very often used in classification.

In [8]:
# This is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):  # Numerically very unstable version
    counts = [logit.exp() for logit in logits]
    denominator = sum(counts)
    out = [count / denominator for count in counts]
    return out


# This is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log()  # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [
    0.041772570515350445,
    0.8390245074625319,
    0.005653302662216329,
    -0.8864503806400986,
]
for dim in range(4):
    ok = "OK" if abs(logits[dim].grad - ans[dim]) < 1e-5 else "WRONG!"
    print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


Verify the gradient using the ``torch`` library. ``torch` should give you the exact same gradient.

In [9]:
logits = torch.tensor([0.0, 3.0, -2.0, 1.0], dtype=torch.double, requires_grad=True)
probs = F.softmax(logits, dim=0)
loss = -probs[3].log()
loss.backward()
torch.set_printoptions(precision=9)
print(logits.grad)


tensor([ 0.041772571,  0.839024507,  0.005653303, -0.886450381],
       dtype=torch.float64)
